I made too much simple NN model.
https://www.kaggle.com/artgor helped me.

I'm beginner, so there may be many strange point.
Please give me a advise.

--------------------------------------------------------------

NNによるシンプルな回帰モデルを構築しました。
https://www.kaggle.com/artgor　さんのkernelsを参考にさせていただきました。

日本語で書かれたkernelsがほとんどなかったため
少しでも初心者の助けになればとこちらのkernelsを作成します。

なお、私自身も初心者ですので理解のできていない点が多くあるかと思います。
ご指摘やアドバイスがあれば是非お願いします。

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, RidgeCV
import gc
from catboost import CatBoostRegressor
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


['test', 'train.csv', 'sample_submission.csv']


Loading Data.

-------------------

データの読み込み。

In [2]:
X = pd.read_csv("../input/train.csv", nrows = 600000000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

Making Features.

-----------------------

特徴量の作成。

In [3]:
%time

rows = 150_000
train = X
segments = int(np.floor(train.shape[0] / rows))

X_tr = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min',
                               'av_change_abs', 'av_change_rate', 'abs_max', 'abs_min',
                               'std_first_50000', 'std_last_50000', 'std_first_10000', 'std_last_10000',
                               'avg_first_50000', 'avg_last_50000', 'avg_first_10000', 'avg_last_10000',
                               'min_first_50000', 'min_last_50000', 'min_first_10000', 'min_last_10000',
                               'max_first_50000', 'max_last_50000', 'max_first_10000', 'max_last_10000'])
y_tr = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

total_mean = train['acoustic_data'].mean()
total_std = train['acoustic_data'].std()
total_max = train['acoustic_data'].max()
total_min = train['acoustic_data'].min()
total_sum = train['acoustic_data'].sum()
total_abs_max = np.abs(train['acoustic_data']).sum()

for segment in tqdm_notebook(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values[-1]
    
    y_tr.loc[segment, 'time_to_failure'] = y
    X_tr.loc[segment, 'ave'] = x.mean()
    X_tr.loc[segment, 'std'] = x.std()
    X_tr.loc[segment, 'max'] = x.max()
    X_tr.loc[segment, 'min'] = x.min()
    
    
    X_tr.loc[segment, 'av_change_abs'] = np.mean(np.diff(x))
    X_tr.loc[segment, 'av_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_tr.loc[segment, 'abs_max'] = np.abs(x).max()
    X_tr.loc[segment, 'abs_min'] = np.abs(x).min()
    
    X_tr.loc[segment, 'std_first_50000'] = x[:50000].std()
    X_tr.loc[segment, 'std_last_50000'] = x[-50000:].std()
    X_tr.loc[segment, 'std_first_10000'] = x[:10000].std()
    X_tr.loc[segment, 'std_last_10000'] = x[-10000:].std()
    
    X_tr.loc[segment, 'avg_first_50000'] = x[:50000].mean()
    X_tr.loc[segment, 'avg_last_50000'] = x[-50000:].mean()
    X_tr.loc[segment, 'avg_first_10000'] = x[:10000].mean()
    X_tr.loc[segment, 'avg_last_10000'] = x[-10000:].mean()
    
    X_tr.loc[segment, 'min_first_50000'] = x[:50000].min()
    X_tr.loc[segment, 'min_last_50000'] = x[-50000:].min()
    X_tr.loc[segment, 'min_first_10000'] = x[:10000].min()
    X_tr.loc[segment, 'min_last_10000'] = x[-10000:].min()
    
    X_tr.loc[segment, 'max_first_50000'] = x[:50000].max()
    X_tr.loc[segment, 'max_last_50000'] = x[-50000:].max()
    X_tr.loc[segment, 'max_first_10000'] = x[:10000].max()
    X_tr.loc[segment, 'max_last_10000'] = x[-10000:].max()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.3 µs


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


Normalize Features.

------------------------

作成した特徴量の正規化。

In [4]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_train_scaled = pd.DataFrame(scaler.transform(X_tr), columns=X_tr.columns)

Making data to predict.

--------------------------------

訓練データの作成。

In [5]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)
plt.figure(figsize=(22, 16))

for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
        
    X_test.loc[seg_id, 'av_change_abs'] = np.mean(np.diff(x))
    X_test.loc[seg_id, 'av_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_test.loc[seg_id, 'abs_max'] = np.abs(x).max()
    X_test.loc[seg_id, 'abs_min'] = np.abs(x).min()
    
    X_test.loc[seg_id, 'std_first_50000'] = x[:50000].std()
    X_test.loc[seg_id, 'std_last_50000'] = x[-50000:].std()
    X_test.loc[seg_id, 'std_first_10000'] = x[:10000].std()
    X_test.loc[seg_id, 'std_last_10000'] = x[-10000:].std()
    
    X_test.loc[seg_id, 'avg_first_50000'] = x[:50000].mean()
    X_test.loc[seg_id, 'avg_last_50000'] = x[-50000:].mean()
    X_test.loc[seg_id, 'avg_first_10000'] = x[:10000].mean()
    X_test.loc[seg_id, 'avg_last_10000'] = x[-10000:].mean()
    
    X_test.loc[seg_id, 'min_first_50000'] = x[:50000].min()
    X_test.loc[seg_id, 'min_last_50000'] = x[-50000:].min()
    X_test.loc[seg_id, 'min_first_10000'] = x[:10000].min()
    X_test.loc[seg_id, 'min_last_10000'] = x[-10000:].min()
    
    X_test.loc[seg_id, 'max_first_50000'] = x[:50000].max()
    X_test.loc[seg_id, 'max_last_50000'] = x[-50000:].max()
    X_test.loc[seg_id, 'max_first_10000'] = x[:10000].max()
    X_test.loc[seg_id, 'max_last_10000'] = x[-10000:].max()
    
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


<Figure size 1584x1152 with 0 Axes>

Trainig and Predict by NN.

--------------------------------------

NNによる学習および予測。

In [6]:
X_test_scaled.shape

(2624, 24)

In [7]:
from keras.layers import Input, Dense
from keras.models import Model
import tensorflow as tf

# This returns a tensor
inputs = Input(shape=(24,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(128, activation='relu')(inputs)
x = Dense(128, activation='relu')(x)
predictions = Dense(1)(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='mse',
              metrics=['accuracy'])
model.fit(X_train_scaled, y_tr,epochs=10)  # starts training

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
4000/4000 [==============================] - 1s 176us/step - loss: 23.0998 - acc: 0.0000e+00
Epoch 2/10
4000/4000 [==============================] - 0s 44us/step - loss: 10.6214 - acc: 0.0000e+00
Epoch 3/10
4000/4000 [==============================] - 0s 40us/step - loss: 9.7376 - acc: 0.0000e+00
Epoch 4/10
4000/4000 [==============================] - 0s 41us/step - loss: 9.2776 - acc: 0.0000e+00
Epoch 5/10
4000/4000 [==============================] - 0s 37us/step - loss: 9.0397 - acc: 0.0000e+00
Epoch 6/10
4000/4000 [==============================] - 0s 36us/step - loss: 8.7724 - acc: 0.0000e+00
Epoch 7/10
4000/4000 [==============================] - 0s 39us/step - loss: 8.6368 - acc: 0.0000e+00
Epoch 8/10
4000/4000 [==============================] - 0s 37us/step - loss: 8.4919 - acc: 0.0000e+00
Epoch 9/10
4000/4000 [==============================] - 0s 37

In [8]:
y_pred_nn = model.predict(X_test_scaled).flatten()

Making data to submit.

---------------------------

提出用データの作成。

In [9]:
sample = pd.read_csv("../input/sample_submission.csv")
sample['time_to_failure'] = y_pred_nn
sample.to_csv('submission.csv',index=False)

Cross Validation.

---------------------------

交差検証。

In [ ]:
%time

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, roc_auc_score

n_estimators = [15,16]
functions = ['relu']
numbers = [64,128]
for funcs in functions:
  for number in numbers:
    for n_est in  n_estimators:
        print(n_est)
        print(number)
        print(funcs)
        cv = KFold(n_splits=5, shuffle=True,random_state=0)
        for train, valid in cv.split(X_train_scaled, y_tr):
            x_train = X_train_scaled.iloc[train]
            x_valid = X_train_scaled.iloc[valid]
            y_train = y_tr.iloc[train]
            y_valid = y_tr.iloc[valid]
        
            inputs = Input(shape=(24,))
            x = Dense(number, activation=funcs)(inputs)
            x = Dense(number, activation=funcs)(x)
            predictions = Dense(1)(x)
            model = Model(inputs=inputs, outputs=predictions)
            model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
                      loss='mse',
                    metrics=['accuracy'])
            model.fit(x_train, y_train, epochs=n_est, verbose=0)
            y_pred_nn = model.predict(x_valid)
            y_pred = y_pred_nn
            print(mean_absolute_error(y_valid, y_pred))  

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.5 µs
15
64
relu
2.3405214526156595
2.3523465314171337
2.3016325594056943
2.31688229776901
2.291593008028064
16
64
relu
2.3444747597632034
2.2942874239290876
2.3678300082913015
2.3267173563425616
2.195877104222692
15
128
relu
2.3052142118814793
2.3352317691500835
2.3367280551733756
2.285286053278569
2.2310413313773156
16
128
relu
2.3019578612367515
2.309181808223173
2.299370191638223
2.250706806736971
